In [52]:
!pip install config

In [53]:
# Import Matplot lib
import os
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
import sys

In [54]:
import pandas as pd
import numpy as np

In [58]:
# Import SQLAlchemy `automap` and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from config import engine

In [62]:
inspector = inspect(engine)
inspector.get_table_names()

['dept_emp', 'departments', 'dept_manager', 'titles', 'employees', 'salaries']

In [61]:
#Code to get column names from any of the tables
columns = inspector.get_columns('titles')
for c in columns:
    print(c['name'], c["type"])

title_id VARCHAR
title VARCHAR


In [ ]:
#Code to get data from the tables
engine.execute('SELECT * FROM salaries LIMIT 10').fetchall()

# Reflect Database and Query 

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
titles = Base.classes.titles
employees = Base.classes.employees
salaries = Base.classes.salaries

In [ ]:
session = Session(engine)

In [ ]:
employees1 = session.query(employees.emp_no, employees.emp_title_id).\
    order_by(employees.emp_no.desc()).all()

In [ ]:
for employee in employees1:
    print(employees1)

In [ ]:
emp_num = [employee[0] for employee in employees1]
emp_title_id = [employee[1] for employee in employees1]

In [ ]:
title1 = session.query(titles.title_id, titles.title).\
    order_by(titles.title_id.desc()).all()

In [ ]:
title_id = [title[0] for title in title1]
title = [title[1] for title in title1]

In [ ]:
salaries1 = session.query(salaries.emp_no, salaries.salary).\
    order_by(salaries.emp_no.desc()).all()

In [ ]:
salary_emp_num = [salary[0] for salary in salaries1]
salary = [salary[1] for salary in salaries1]

# Adding lists to pandas DF

In [ ]:
emp_df = pd.DataFrame(emp_num, emp_title_id, columns = ['emp_num'])
emp_df['row_num'] = np.arange(len(emp_df))
emp_df.set_index(['row_num'], inplace=True)
emp_df['emp_title_id'] = emp_title_id
emp_df

In [ ]:
title_df = pd.DataFrame(title_id, title, columns = ['title_id'])
title_df['row_num'] = np.arange(len(title_df))
title_df.set_index(['row_num'], inplace=True)
title_df['title'] = title
title_df

In [ ]:
salary_df = pd.DataFrame(salary_emp_num, salary, columns = ['emp_num'])
salary_df['row_num'] = np.arange(len(salary_df))
salary_df.set_index(['row_num'], inplace=True)
salary_df['salary'] = salary
salary_df
salary_df

In [ ]:
emp_salary = emp_df.merge(salary_df, on ='emp_num')

In [ ]:
emp_salary_title = emp_salary.merge(title_df, left_on='emp_title_id', right_on='title_id')

In [ ]:
emp_salary_title

In [ ]:
title_salaries = emp_salary_title.groupby('title')['salary'].mean().round(2)

In [ ]:
title_salaries

In [ ]:
title_salaries.plot.bar()
plt.xlabel("Job Title")
plt.ylabel("Average Salary")